# Exportando Modelo a ONNX

In [1]:
from src import *

In [2]:
import wandb

In [3]:
wandb.init(mode="offline")
module = MNISTModule.load_from_checkpoint('checkpoints/008-val_loss=0.19057-epoch=5.ckpt')
dm = MNISTDataModule(**module.hparams['datamodule'])
dm.setup()
print(module.hparams)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


"callbacks":     [{'name': 'WandBCallback', 'lib': 'src.utils', 'params': {'labels': ['no 3', '3'], 'dl': <torch.utils.data.dataloader.DataLoader object at 0x0000021760C0D590>}}, {'name': 'ModelCheckpoint', 'lib': 'pytorch_lightning.callbacks', 'params': {'dirpath': 'checkpoints', 'filename': '008-{val_loss:.5f}-{epoch}', 'save_top_k': 1, 'monitor': 'val_loss', 'mode': 'min'}}]
"datamodule":    {'path': WindowsPath('dataset'), 'batch_size': 25}
"logger":        WandbLogger
"logger_params": {'project': 'dlops-mnist', 'name': '008'}
"trainer":       {'max_epochs': 10, 'enable_checkpointing': True, 'overfit_batches': 0, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x0000021760BF9790>, 'callbacks': [<src.utils.WandBCallback object at 0x0000021760C49490>, <pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000021760BD7B90>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x0000021760C4AB10>, <pytorch_lightning.callback

c:\Users\robme\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


In [4]:
import torch

def torch_eval():
    module.eval()
    with torch.no_grad():
        preds, labels = torch.tensor([]), torch.tensor([])
        for imgs, _labels in dm.val_dataloader():
            outputs = module.predict(imgs.to(module.device)) > 0.5
            preds = torch.cat([preds, outputs.cpu().long()])
            labels = torch.cat([labels, _labels])

    acc = (preds == labels).float().mean()
    return acc.item()

torch_eval()

0.9312499761581421

Ahora exportaremos un modelo a ONNX

In [5]:
import onnx
import pytorch_lightning

Tuve que hacer pip install onnx. Luego de ello reiniciar kernel de este Notebook para que los cambios se hicieran efectivos.

In [6]:
input_sample = torch.randint(0, 255, (1, 28, 28), dtype=torch.uint8)
module.to_onnx(
    'models/binary_classifier_3.onnx',
    input_sample,
    export_params=True,
    opset_version=11,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0 : 'batch_size'},
        'output' : {0 : 'batch_size'},
    }
)

In [7]:
import onnxruntime as ort
import numpy as np

ort_session = ort.InferenceSession('models/binary_classifier_3.onnx')

ort_inputs = {
    "input": np.random.randint(0, 255, (10, 28, 28), dtype=np.uint8),
}

ort_output = ort_session.run(['output'], ort_inputs)
ort_output[0].shape

(10,)

Es importante evaluar el modelo para asegurarnos que lo hemos exportado bien

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def onnx_eval():
    with torch.no_grad():
        preds, labels = [], torch.tensor([])
        for imgs, _labels in dm.val_dataloader():
            ort_inputs = {
                "input": imgs.numpy(),
            }
            ort_output = ort_session.run(['output'], ort_inputs)[0]
            outputs = sigmoid(ort_output) > 0.5
            preds += outputs.astype(int).tolist()
            labels = torch.cat([labels, _labels])
    acc = (np.array(preds) == labels.numpy()).astype(float).mean()
    return acc

onnx_eval()

0.93125

Coincide el valor con el del checkpoint por lo que debiera andar bien el modelo exportado.

# Versionando Modelos

Luego de haber exportado el modelo, y verificado que funciona correctamente entonces se versionará con DVC.

In [9]:
# dvc add models
# dvc push

Para recuperar este modelo

In [10]:
# dvc pull models.dvc

Eliminé la carpeta models y luego de eso ejecuté "dvc pull models.dvc" (tal cual). Esto recuperó la carpeta models y la devolvió a mi sector de trabajo :)

Y recordar generar un nuevo tag y sincronizar con el repositorio de git

In [ ]:
# git add .
# git commit -m "primer modelo"
# git push
# git tag -a v3 -m "version 3"
# git push origin --tags